In [2]:
#import libraries / dependencies
import openpyxl
from openpyxl import load_workbook, Workbook
import csv
from pyexcel.cookbook import merge_all_to_a_book, merge_csv_to_a_book
import os
import re

In [4]:
#FUNCTIONS BLOCK

#get_filenames: 
#Arguments: raw data filepath (absolute)
#Returns: A tuple of strings representing the files that will be used to 
#represent the data at each step of refinement
def get_filenames(rdfile):
    #EXAMPLE '/Users/default/Downloads/Videology Reports/TruOptik - Monthly Usage - June 2018.xlsx'
    rd = rdfile.split('/')
    path = '/'.join(rd[:-1])
    extractedfile = path+'/Extracted-'+rd[-1][:-4]+'csv'
    correctedfile = path+'/Corrected-'+rd[-1][:-4]+'csv'
    finalfile = path+'/Final-'+rd[-1]
    correctedfile_excel = path+'/Corrected-'+rd[-1]
    return extractedfile, correctedfile, finalfile, correctedfile_excel

get_filenames('/Users/default/Downloads/Videology Reports/TruOptik - Monthly Usage - June 2018.xlsx')

#replacements:
#Argument: the videology category (string)
#Returns: the category with all necessary corrections (string)
#Basically there are duplicates in the categories that are *slightly* different
#and this takes care of those
def replacements(category):
    category = category.replace('Financial','Financial/Insurance Attributes & Behaviors')
    category = category.replace('Financial/Insurance Attributes & Behaviors/Insurance Attributes & Behaviors','Financial/Insurance Attributes & Behaviors')
    category = category.replace('"','')
    category = category.replace('Home & Garden','Home Imp/Décor/Home & Garden/Home&Fam/DIY').replace('Home Imp/Décor/Home Imp/Décor/Home & Garden/Home&Fam/DIY/Home&Fam/DIY','Home Imp/Décor/Home & Garden/Home&Fam/DIY')
    category = category.replace('Demos','Demo')
    category = category.replace('Sports & Outdoors','Sporting & Healthy Living + Sporting Goods/Outdoor')
    category = category.replace('Education/Career','Education & Career')
    category = category.replace('Entertainment/Media','Entertainment & Media')
    category = category.replace('Fashion & Style', 'Fashion/Style; Apparel; Accessories')
    category = category.replace('Health','Health & Fitness/Wellness').replace('Health & Fitness/Wellness & Fitness/Wellness','Health & Fitness/Wellness')
    return category

In [61]:
#BLOCK 1: EXTRACT THE ATTRIBUTES, IDS, AND USAGE DATA FROM THE EXCEL SHEET and get unique values. Then store in CSV

#absolute filepath to raw_data (Videology Report)
raw_data = '/Users/default/Downloads/Videology Reports/filedump/TruOptik - Monthly Usage - April 2018.xlsx'

#This program will create the edited files in the same location as the original, so look there for the data
extractedfile, correctedfile, finalfile, correctedfile_excel = get_filenames(raw_data)

#use openpyxl library to create a workbook object from the xlsx file so we can work w it
rd_wb = load_workbook(raw_data)

#get the worksheet with the data
#usually the sheet is named 'Behavioral Cost', but sometimes there is a trailing space or its named something else
#because Videology is allergic to consistency. Either way changing the string will get you the desired worksheet
rd_ws = rd_wb['Behavioral Cost']

#the header has the names for each column in the worksheet
header = [x.value for x in list(rd_ws.rows)[0]]
print(header)

#we want the index of the columns for the following: 
#Retargeting Attribute
#Attribute ID (sometimes called External Retargeting ID so just change string)
#Usage (sometimes called Billable Usage)
ra_ind = header.index('Retargeting Attribute')
erid_ind= header.index('Attribute ID')
usage_ind = header.index('Usage')

print([ra_ind, erid_ind, usage_ind])

#The following block of code creates a dictionary of unique attributes
#Dictionary format is "unique_records[attribute] = usage"

unique_records = {}
for item in [[i.value for i in x] for x in rd_ws.iter_rows(min_row=2)]:
    #edit the row if it exists
    if item[ra_ind] is not None: 
        row = [item[ra_ind].replace(',',';').replace('  ',' '),item[erid_ind],item[usage_ind]]
    else:
        pass
    #check for membership in the dictionary keys
    if row[0] in list(unique_records.keys()):
        #if the usage value is an int, add it to the dictionary (this gets rid of excel formulas)
        if isinstance(item[usage_ind],int):
            unique_records[row[0]] += int(item[usage_ind])
    #otherwise create new entry
    elif row[0] not in list(unique_records.keys()):
        unique_records[row[0]] = int(item[usage_ind])

#write this unique record data to a csv
extracted_data = open(extractedfile,'w')
for kv in unique_records.items():
    write = csv.writer(extracted_data)
    write.writerow(kv)
extracted_data.close()

['Provider', 'Country', 'Retargeting Attribute', 'Attribute ID', 'Device Type', 'Format Type', 'CPU', 'Usage', 'Data Costs', 'Rate Adjustment', 'Adjusted Cost (PC/Mobile)']
[2, 3, 7]


In [36]:
#BLOCK 1A: IF THE DATA ISN'T IN A FRIENDLY FORMAT IN THE RAW DATA, INSTEAD USE A SLIGHTLY EDITED VERSION AND CONTINUE FROM THERE

refined_data = '/Users/default/Documents/PythonMisc/Videology Reports - Completed/Edited - Dec2017Categories copy.xlsx'
wb = load_workbook(refined_data)
ws = wb.active
category_dict = {}
for line in [[i.value for i in x] for x in ws.iter_rows()]:
    if line[1].strip() in list(category_dict.keys()):
        category_dict[line[1].strip()] += int(line[2])
    else:
        category_dict[line[1].strip()] = int(line[2])
for key, value in category_dict.items():
    index = list(category_dict.keys()).index(key) + 1
    ws.cell(row = index, column = 4).value = key
    ws.cell(row = index, column = 5).value = value
wb.save(refined_data)

In [43]:
#BLOCK 2: CATEGORIZE THE UNIQUE VALUES USING PREVIOUS DATA
#load reference file and create a list of lists of its values

db_path = '/Users/default/Documents/PythonMisc/VideologyDatabase.xlsx'
db_file = load_workbook(db_path)
db_ws = db_file.active
db_contents = {}
for attribute, category in [[i.value for i in x] for x in db_ws.iter_rows(max_col=2)]:
    db_contents[attribute] = category
    
print(len(db_contents))
#load file with missing categories and create list of values
edit_file = open(extractedfile,'r')
corrected_vals = []
for line in edit_file:
    attribute = line.replace('"','').strip().split(',',1)[0]
    print(attribute)
    usage = line.replace('"','').strip().split(',',1)[1]
    try:
        if 'hhi' in attribute.lower() or 'income' in attribute.lower():
            corrected_vals.append([attribute,'Income', usage])
        else:
            corrected_vals.append([attribute,db_contents[attribute], usage])
        
    except KeyError:
        corrected_vals.append([attribute,'',usage])
    '''
    TEMPORARY CHANGE
    if attribute in list(ref_data.keys()):
        corrected_vals.append([attribute, db_contents[attribute],usage])
    elif attribute not in list(ref_data.keys()):
        corrected_vals.append([attribute,'',usage])
    '''
corrected_file = open(correctedfile,'w')
for value in corrected_vals:
    write = csv.writer(corrected_file)
    write.writerow(value)
corrected_file.close()

1410
Tru Optik > V12Data > Auto Budget > Auto Budget Range <= $29K
Tru Optik > V12Data > Auto Budget > Auto Budget Range $30k - $49k
Tru Optik > V12Data > Auto Budget > Auto Budget Range >= $50k
Tru Optik > V12Data > In Auto Market Predictor > Truck; Van
Tru Optik > V12Data > In Auto Market Predictor > Luxury; Full Size
Tru Optik > V12Data > In Auto Market Predictor > Mid Size Car
Tru Optik > V12Data > In Auto Market Predictor > Small Car
Tru Optik > V12Data > Baby and Toddlers > Baby Care Accessories ; Baby Food & Bev; Baby Formula; Toddler Diapers
Tru Optik > V12Data > OTC Medicine/Cough & Cold > OTC - Pain Relievers (Kids; Brand); Cough & Cold (Kids; Brand)
Tru Optik > V12Data > Income > Income Range: K-$100K-$149.9K
Tru Optik > V12Data > Financial > Owns Mutual Funds; Stocks; or Bonds
Tru Optik > V12Data > Financial > Credit Card Holder
Tru Optik > V12Data > Health & Beauty > Women's Health & Beauty Aids
Tru Optik > V12Data > Health & Beauty > Men's Health & Beauty Aids
Tru Optik >

In [45]:
#BLOCK 3: CREATE DICTIONARY BASED ON UNIQUE CATEGORIES AND WRITE TO XL FILE
#ONLY RUN THIS BLOCK AFTER EDITS HAVE BEEN MADE TO CSV CREATED IN PREVIOUS BLOCK

#convert corrected csv to excel file so we can write into it
merge_csv_to_a_book([correctedfile],correctedfile_excel)


#create dictionary
category_dict = {}
corrected_csv = open(correctedfile)
for line in corrected_csv:
    line = line.replace('"""Fashion/Style, Apparel, Accessories"""', ',Fashion/Style; Apparel; Accessories').replace(',,',',').strip()
    if len(line.split(',')) > 3:
        line = line.replace('Style,','Style;').replace('Apparel,','Apparel;')
    line = line.split(',')
    attribute, category=line[:2]
    category = replacements(category)
    
    usage=int(line[2])
    if category in list(category_dict.keys()):
        category_dict[category] += usage
    elif category not in list(category_dict.keys()):
        category_dict[category] = usage
    else:
        print('sumthin fucky')

wb = load_workbook(correctedfile_excel)
ws = wb.active
for key, value in category_dict.items():
    index = list(category_dict.keys()).index(key)+1
    ws.cell(row = index, column = 4).value = key
    ws.cell(row = index, column = 5).value = value

wb.save(finalfile)


/Users/default/anaconda/lib/python3.6/site-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [39]:
#SIDE PROJECT: CREATE A DATABASE OF VALUES THAT HAVE CORRECT LABELS
#RUN ONLY ONCE: THIS WILL OVERWRITE THE CURRENT DATABASE
folder_path = '/Users/default/Documents/PythonMisc/Videology Reports - Completed'
folder_contents = os.listdir(folder_path)
#all files are excel, openpyxl will be used

#also, we have to load our database so we know what we have/don't
db_path = '/Users/default/Documents/PythonMisc/VideologyDatabase.xlsx'
db_file = load_workbook(db_path)
db_ws = db_file.active
db_contents = {}
pattern = '^[Ee]'

print(len(db_contents))
for attribute, category in [[i.value for i in x] for x in db_ws.iter_rows(max_col=2)]:
    db_contents[attribute] = category

for file in folder_contents:
    if re.match(pattern, file):
        wb = load_workbook(folder_path + '/' + file)
        ws = wb.active

        #only want the first 2 columns (attribute and category)
        for attribute, category in [[i.value for i in x] for x in ws.iter_rows(max_col=2)]:
            if attribute is None:
                attribute = 'TEST'
                category = 'TEST'
            if '>' == attribute[-1]:
                attribute = attribute[:-2]
            try:
                category = replacements(category)
            except:
                print('no bueno')
            db_contents[attribute] = category
            '''
            if attribute in list(db_contents.keys()):
                pass
            elif attribute not in list(db_contents.keys()):
                db_contents[attribute] = category
            '''
print(len(db_contents))
#Now, we write the completed dictionary to the database file again
for i, (key, value) in enumerate(db_contents.items()):
    db_ws.cell(row=i+1, column = 1).value = key
    db_ws.cell(row=i+1, column = 2).value = value
    
db_file.save(db_path)

#pattern = '^[Ee]'
#for file in folder_contents:
#    if re.match(pattern, file):
#        print(file)


0


NameError: name 'pattern' is not defined

In [59]:
#FIX THE 2017 REPORTS
folder = '/Users/default/Documents/PythonMisc/Videology Reports - Completed'
for testfile in [x for x in os.listdir(folder) if x[0] != '.' and x[:2] != '~$' and x[0] != '2']:
    #print(testfile)
    exp_testfile = '/'.join(testfile.split('/')[:-1])+'/Experimental-'+testfile.split('/')[-1]
    #print(folder+'/'+testfile[2:])
    if '~$' in testfile:
        print(testfile)
        testfile = testfile[15:]
    wb = load_workbook(folder+'/'+testfile)
    ws = wb.active
    cdict = {}

    for i,row in enumerate(ws.iter_rows(max_col = 3),start=1):
        attr, category, usage = [x.value for x in row]
        if all([attr,category,usage]):
            category = replacements(category)
            if 'hhi' in attr.lower() or 'income' in attr.lower():
                category = 'Income'
                ws.cell(row=i,column=2).value = 'Income'
            if category in list(cdict.keys()):
                cdict[category] += usage
            elif category not in list(cdict.keys()):
                cdict[category] = usage
        else:
            print('One of the values is None')
    for i,(key, value) in enumerate(cdict.items(),start=1):
        ws.cell(row = i, column = 6).value = key
        ws.cell(row = i, column = 7).value = value
        ##print(ws.cell(row = i, column = 6).value)
        ##print(ws.cell(row = i, column = 7).value)

    wb.save(folder+exp_testfile)

/Users/default/anaconda/lib/python3.6/site-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [54]:
folder = '/Users/default/Documents/PythonMisc/Videology Reports - Completed/Correct Category Values 2017'
for file in os.listdir(folder):
    print(file)
    print(file[13:])
    os.rename(folder+'/'+file, folder+'/'+file[13:])
    
    #os.rename(folder+'/'+file, folder+'/'+file[11:])

Experimental-Edited - August2017TestData.xlsx
Edited - August2017TestData.xlsx
Experimental-Edited - July2017TestDat.xlsx
Edited - July2017TestDat.xlsx
Experimental-Edited - Dec2017Categories copy.xlsx
Edited - Dec2017Categories copy.xlsx
Experimental-Edited - Nov2017TestData.xlsx
Edited - Nov2017TestData.xlsx
Experimental-Edited - Sept2017TestData.xlsx
Edited - Sept2017TestData.xlsx
Experimental-Edited - Oct2017TestData.xlsx
Edited - Oct2017TestData.xlsx


In [15]:
#folder = '/Users/default/Documents/PythonMisc/Videology Reports - Completed'
#files = [file for file in os.listdir(folder) if '2017' in file]

# for file in files:
#     print(file)
#     wb = load_workbook(folder+'/'+file)
#     ws = wb.active
#     cdict = {}   
#     for i,row in enumerate([[i.value for i in x[:3]] for x in ws.iter_rows()], start=1):
#         #print([x[0].value for x in ws.iter_rows()].index(row[0]))
#         if row[1]:
#             row[1] = replacements(row[1]) 
#         if row[0] and ('hhi' in row[0].lower() or 'income' in row[0].lower()):
#             ws.cell(row=i, column=2).value = 'Income'
#             #print(ws.cell(row=i, column=2).value)
#             row[1] = 'Income'
#     for category, usage in [[i.value for i in x[1:3]] for x in ws.iter_rows()]:
#         if category and usage:
#             print(category)
#             if category in list(cdict.keys()):
#                 cdict[category] += usage
#             elif category not in list(cdict.keys()):
#                 cdict[category] = usage
#         else:
#             pass   
#     print(len(cdict))
#     wb.save(folder+'/Experimental'+file)

Edited - Oct2017TestData.xlsx
Edited - Dec2017Categories copy.xlsx
Edited - Sept2017TestData.xlsx
Edited - Nov2017TestData.xlsx
Edited - August2017TestData.xlsx
Edited - July2017TestDat.xlsx
/Users/default/Documents/PythonMisc/Videology Reports - Completed/Experimental-Edited - July2017TestDat.xlsx
One of the values is None
One of the values is None
One of the values is None
('Auto', 35113)
Auto
35113
('Health & Fitness/Wellness', 854)
Health & Fitness/Wellness
854
('Home Imp/Décor/Home & Garden/Home&Fam/DIY', 9863)
Home Imp/Décor/Home & Garden/Home&Fam/DIY
9863
('Demo', 482287)
Demo
482287
('Entertainment & Media', 35375)
Entertainment & Media
35375
('Retail', 243452)
Retail
243452
('Career & Education', 125678)
Career & Education
125678


In [102]:
#  SIDE PROJECT #2: CREATE SEPARATE CATEGORY FOR INCOME VALUES AND WRITE THOSE TO THE EXCEL FILE
folder = '/Users/default/Documents/PythonMisc/Videology Reports - Completed'
files = [file for file in os.listdir(folder) if file[0] != '.']
for file in files:
    print(file)
    wrkbk = load_workbook(folder+'/'+file)
    wsht = wrkbk.active
    for i,row in enumerate(wsht.iter_rows(max_col=3)):
        try:
            category = row[1].value.lower()
        except:
            pass
        if 'hhi' in category or 'income' in category:
            wsht.cell(row=i+1, column=2).value = 'Income'
    
    
    wrkbk.save(folder+'/'+file)


ExperimentalEdited - Sept2017TestData.xlsx
7
Edited - Oct2017TestData.xlsx
7
ExperimentalEdited - Nov2017TestData.xlsx
7
Edited - MarchCategories.xlsx
7
Edited - May 2018Categories.xlsx
7
ExperimentalEdited - July2017TestDat.xlsx
7
Edited - Dec2017Categories copy.xlsx
7
Edited - Sept2017TestData.xlsx
7
Edited - Nov2017TestData.xlsx
7
ExperimentalEdited - Oct2017TestData.xlsx
7
Edited - JanCategories.xlsx
7
Edited - August2017TestData.xlsx
7
ExperimentalEdited - Dec2017Categories copy.xlsx
7
ExperimentalEdited - August2017TestData.xlsx
7
Final - June2018Categories.xlsx
7
Edited - TruOptik - Monthly Usage - May 2018.xlsx
7
Edited - July2017TestDat.xlsx
7
Edited - AprilCategories.xlsx
7
Edited - FebCategories.xlsx
7
Edited - June2018Categories.xlsx
7


/Users/default/anaconda/lib/python3.6/site-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
